In [ ]:
pip install pokebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import pandas as pd
import pokebase as pb
import requests
import numpy as np
from requests import get
from bs4 import BeautifulSoup
import re
from datetime import datetime

In [ ]:
#Pedimos el número total de pokemonos
number = requests.get('https://pokeapi.co/api/v2/pokemon?limit=100000&offset=0')
number = number.json()
n=number['count']
n

1154

In [ ]:
#Pedimos el número total de generaciones
number2 =requests.get('https://pokeapi.co/api/v2/generation')
number2 = number2.json()
n2=number2['count']
n2

8

In [ ]:
#Una opción para obtener los pokemons https://pokeapi.co/api/v2/pokemon?limit=100000&offset=0
#Otra https://pokeapi.co/api/v2/generation/1
#Por número https://pokeapi.co/api/v2/pokemon-species/2
link='https://pokeapi.co/api/v2/'

In [ ]:
#Obtener los movimientos de los pokemon
ind2 = requests.get(link+'pokemon/'+str(800))
ind2 = ind2.json()
str(ind2['moves'][0]['move']['name'])

'swords-dance'

In [ ]:
from pandas.core.indexes.extension import Index
class get_pokemon:

  def __init__(self, numero):
    ind = requests.get(link+'pokemon-species/'+str(numero+1)+'/')
    ind2 = requests.get(link+'pokemon/'+str(numero+1))
    ind = ind.json()
    ind2 = ind2.json()
    self.id= ind['id']
    self.name= ind['name']
    self.gen= ind['generation']['name']
    self.type=listas(ind2, 'types', 'type', 'name')
    self.abilities=listas(ind2,'abilities', 'ability', 'name')
    self.capture_rate=ind['capture_rate']
    self.base_happiness=ind['base_happiness']
    self.version_debut=''
    self.moves=ind2['moves'][0]['move']['name']
    self.attack=ind2['stats'][1]['base_stat']
    self.defense=ind2['stats'][2]['base_stat']
    self.hp=ind2['stats'][0]['base_stat']
    self.sp_attack=ind2['stats'][3]['base_stat']
    self.sp_defense=ind2['stats'][4]['base_stat']
    self.speed=ind2['stats'][5]['base_stat']

    if str(ind2['moves'])=='[]':
      self.moves= np.nan
    else:
      self.moves=ind2['moves'][0]['move']['name']


    #Obtenemos la clase de pokemon
    if ind['is_baby']==True:
      self.clase='Baby'
    elif ind['is_legendary']==True:
      self.clase='Legendary'
    elif ind['is_mythical']==True:
      self.clase='Mythical'
    else:
      self.clase='Común'


In [ ]:
def listas(dataf, name, dato, dato2):
  k=dataf[name]
  lista=[]
  if dato2!=True:
    for i in range(len(k)):
      lista.append(k[i][dato][dato2])
  else:
    for i in range(len(k)):
      lista.append(k[i][dato])
  if len(lista)==1:
    lista.append('')
    return lista[0]
  else:
    lista=sorted(lista)
    return lista

In [ ]:
data=[]
for i in range(898):
  datos={}
  pokemon=get_pokemon(i)
  datos['ID']=pokemon.id
  datos['Gen']=pokemon.gen
  datos['Name']=pokemon.name
  datos['Type']=pokemon.type
  data.append(datos)
  if i%100==0:
    print(i)

0
100
200
300
400
500
600
700
800


# Tabla de Pokémon

Crea tabla para regiones(para agregar las regiones)

In [ ]:
#Se creo esta tabla para agregar las regiones de acuerdo a las 8 generaciones.  #--copia inicio
G1=[1,'kanto']
G2=[2,'johto']
G3=[3,'hoenn']
G4=[4,'sinnoh']
G5=[5,'unova']
G6=[6,'kalos']
G7=[7,'alola']
G8=[8,'galar']

lista_reg=[G1,G2,G3,G4,G5,G6,G7]
df_region=pd.DataFrame(lista_reg,columns=['Gen','Region'])
# copia final

In [ ]:
generaciones={'generation-i':1, 'generation-ii':2, 'generation-iii':3, 'generation-iv':4,
       'generation-v':5, 'generation-vi':6, 'generation-vii':7,
       'generation-viii':8}

In [ ]:
tabla1=pd.DataFrame(data)
tabla1['Gen'].replace(generaciones, inplace= True)
tabla1

,ID,Gen,Name,Type
0,1,1,bulbasaur,"[grass, poison]"
1,2,1,ivysaur,"[grass, poison]"
2,3,1,venusaur,"[grass, poison]"
3,4,1,charmander,fire
4,5,1,charmeleon,fire
...,...,...,...,...
893,894,8,regieleki,electric
894,895,8,regidrago,dragon
895,896,8,glastrier,ice
896,897,8,spectrier,ghost


In [ ]:
new1 = tabla1.set_index('ID')

In [ ]:
new1.to_csv('tabla_de_pokemon')

# Tabla Características

Se van a juntar la tabla de caracteristicas y de region para generar la Tabla de caracteristicas que ocupamos con el dato de region.

In [ ]:
data1=[]
for i in range(898):
  datos1={}
  pokemon=get_pokemon(i)
  datos1['ID']=pokemon.id
  datos1['Clase']=pokemon.clase
  datos1['TasaDeCaptura']=pokemon.capture_rate
  datos1['Felicidad']=pokemon.base_happiness
  datos1['Habilidades']=pokemon.abilities
  datos1['Gen']=pokemon.gen #--copia
  data1.append(datos1)
  if i%100==0:
    print(i)
tabla2=pd.DataFrame(data1)
tabla2['Gen'].replace(generaciones, inplace= True) #copia
tabla2_p1=pd.merge(tabla2 ,df_region, on='Gen')#copia
#tabla2_final=tabla2_n.drop(['Gen'],axis=1)#copia
#copia (solo para visualizar la tabla)


0
100
200
300
400
500
600
700
800


# WebScraping para tabla de características

In [ ]:
url="https://press.pokemon.com/en/Items/Schedule/Video-Game?types=2"

In [ ]:
response=get(url)
mysoup=BeautifulSoup(response.text,'html.parser')

In [ ]:
mysoup.find('p')
mysoup.find('div')
mysoup.find(id='p')
tabla=mysoup.find_all("div",class_="tableScheduleContainer")
tabla_pok=mysoup.find("div",id="footerBg")
tabla_pok=mysoup.find("div",class_="tableScheduleContainer")
tabla_pok=str(tabla_pok)
tabla_pok

'<div class="tableScheduleContainer">\n<div class="loadingOuter big jq-loading"><div class="loadingBg"><div class="loadingInner"></div></div></div>\n<table class="tableSchedule table table-hover jq-table-row-click jq-scheduleHistory">\n<thead>\n<tr>\n<th class="th-prod">\n<a class="sortable jq-sortable" data-sortby="Product" href="#">\r\n            Product\r\n\t\t\t<span class="sort">\n<span aria-hidden="true" class="sort-none fa fa-sort"></span>\n<span aria-hidden="true" class="sort-asc fa fa-sort-alpha-asc"></span>\n<span aria-hidden="true" class="sort-desc fa fa-sort-alpha-desc"></span>\n</span>\n</a>\n</th>\n<th class="th-plat">\n<a class="sortable jq-sortable" data-sortby="Format" href="#">\n<span>Available Platforms</span>\n<span class="sort">\n<span aria-hidden="true" class="sort-none fa fa-sort"></span>\n<span aria-hidden="true" class="sort-asc fa fa-sort-alpha-asc"></span>\n<span aria-hidden="true" class="sort-desc fa fa-sort-alpha-desc"></span>\n</span>\n</a>\n</th>\n<th cla

In [ ]:
tabla_pok=re.sub(r'<.*?>',lambda g:g.group(0).upper(),tabla_pok)

In [ ]:
df=pd.read_html(tabla_pok)[0]


,Product,Available Platforms,Release date
0,Pokémon Scarlet and Pokémon Violet,Nintendo Switch,2022
1,Pokémon Legends: Arceus,Nintendo Switch,2022
2,Pokémon Brilliant Diamond and Pokémon Shining ...,Nintendo Switch,2021
3,Pokémon UNITE,"Android, iOS, Nintendo Switch",Jul 2021
4,New Pokémon Snap,Nintendo Switch,"April 30, 2021"
5,Pokémon Sword Expansion Pass and Pokémon Shiel...,Nintendo Switch,Jun 2020
6,Pokémon Café Mix,"Android, iOS, Nintendo Switch","June 24, 2020"
7,Pokémon Mystery Dungeon: Rescue Team DX,Nintendo Switch,"March 6, 2020"
8,Pokémon Sword and Pokémon Shield,Nintendo Switch,"November 15, 2019"
9,"Pokémon: Let's Go, Pikachu! and Pokémon: Let's...",Nintendo Switch,"November 16, 2018"


In [ ]:
tabla_fechas=df.drop([19],axis=0)


In [ ]:
tabla_fechas.insert(3,"Gen",[8,8,8,8,8,8,8,7,7,7,6,5,5,4,4,4,2,2,1])
new_table=tabla_fechas.drop(["Product","Available Platforms"],axis=1)
new_table.loc[19]=[ 'November 11,2022',3]


In [ ]:
new_table=new_table.drop([1, 2, 3,4,5,7,8,11,13,14,16],axis=0)
new_table

,Release date,Gen
0,2022,8
6,"June 24, 2020",8
9,"November 16, 2018",7
10,"May 29, 2018",6
12,"November 17, 2017",5
15,"November 20, 2015",4
17,"October 25, 1999",2
18,"September 28, 1998",1
19,"November 11,2022",3


Funcionamos ambas tablas

In [ ]:
tabla2_final=pd.merge(tabla2_p1 ,new_table, on='Gen')
tabla2_final=tabla2_final.drop(['Gen'],axis=1)

In [ ]:
new2 = tabla2_final.set_index('ID')
new2

,Clase,TasaDeCaptura,Felicidad,Habilidades,Region,Release date
ID,,,,,,
1,Común,45,50,"[chlorophyll, overgrow]",kanto,"September 28, 1998"
2,Común,45,50,"[chlorophyll, overgrow]",kanto,"September 28, 1998"
3,Común,45,50,"[chlorophyll, overgrow]",kanto,"September 28, 1998"
4,Común,45,50,"[blaze, solar-power]",kanto,"September 28, 1998"
5,Común,45,50,"[blaze, solar-power]",kanto,"September 28, 1998"
...,...,...,...,...,...,...
805,Común,30,0,beast-boost,alola,"November 16, 2018"
806,Común,30,0,beast-boost,alola,"November 16, 2018"
807,Mythical,3,0,volt-absorb,alola,"November 16, 2018"


In [ ]:
new2.to_csv('tabla_caractiristicas')

In [ ]:
tabla2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 898 entries, 0 to 897
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             898 non-null    int64 
 1   Clase          898 non-null    object
 2   TasaDeCaptura  898 non-null    int64 
 3   Felicidad      898 non-null    int64 
 4   Habilidades    898 non-null    object
 5   Gen            898 non-null    int64 
dtypes: int64(4), object(2)
memory usage: 42.2+ KB


# Tabla Estadísticas

In [ ]:
data2=[]
for i in range(898):
  datos2={}
  pokemon=get_pokemon(i)
  datos2['ID']=pokemon.id
  datos2['Movimiento']=pokemon.moves
  datos2['Ataque']=pokemon.attack
  datos2['Defensa']=pokemon.defense
  datos2['HP']=pokemon.hp
  datos2['Ataque_especial']=pokemon.sp_attack
  datos2['Defensa_especial']=pokemon.sp_defense
  datos2['Velocidad']=pokemon.speed
  data2.append(datos2)
  if i%100==0:
    print(i)

tabla3=pd.DataFrame(data2)
tabla3

0
100
200
300
400
500
600
700
800


,ID,Movimiento,Ataque,Defensa,HP,Ataque_especial,Defensa_especial,Velocidad
0,1,razor-wind,49,49,45,65,65,45
1,2,swords-dance,62,63,60,80,80,60
2,3,swords-dance,82,83,80,100,100,80
3,4,mega-punch,52,43,39,60,50,65
4,5,mega-punch,64,58,58,80,65,80
...,...,...,...,...,...,...,...,...
893,894,body-slam,100,50,80,100,50,200
894,895,vice-grip,100,50,200,100,50,80
895,896,swords-dance,145,130,100,65,110,30
896,897,stomp,65,60,100,145,80,130


In [ ]:
new3 = tabla3.set_index('ID')
new3

,Movimiento,Ataque,Defensa,HP,Ataque_especial,Defensa_especial,Velocidad
ID,,,,,,,
1,razor-wind,49,49,45,65,65,45
2,swords-dance,62,63,60,80,80,60
3,swords-dance,82,83,80,100,100,80
4,mega-punch,52,43,39,60,50,65
5,mega-punch,64,58,58,80,65,80
...,...,...,...,...,...,...,...
894,body-slam,100,50,80,100,50,200
895,vice-grip,100,50,200,100,50,80
896,swords-dance,145,130,100,65,110,30


In [ ]:
new3.to_csv('tabla_estadistica')

In [ ]:
tabla3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 898 entries, 0 to 897
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ID                898 non-null    int64 
 1   Movimiento        898 non-null    object
 2   Ataque            898 non-null    int64 
 3   Defensa           898 non-null    int64 
 4   HP                898 non-null    int64 
 5   Ataque_especial   898 non-null    int64 
 6   Defensa_especial  898 non-null    int64 
 7   Velocidad         898 non-null    int64 
dtypes: int64(7), object(1)
memory usage: 56.2+ KB
